## 自动求导

假设对函数 y = 2x^Tx 关于列向量x 求导  
计算 y 关于 X 的梯度： 
1. 存储梯度
2. 调用反向传播函数来自动计算y关于x每个分量的梯度

In [45]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [46]:
x.requires_grad_(True) # 可以与上一步合并，通过传入参数(requires_grad = Ture)实现
x.grad  # 默认值为None

In [47]:
# 计算y
y = 2 * torch.dot(x, x)


In [48]:
# 反向传播求梯度
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [49]:
# 验证
x.grad == 4*x

tensor([True, True, True, True])

**默认情况下，pytorch 会累积梯度**  
当计算另一个函数的时候需要清除累计梯度值  
x.grad.zero_()

In [50]:
x.grad.zero_()
y = x.sum()
y.backward
x.grad

tensor([0., 0., 0., 0.])

深度学习中，目的不是计算微分矩阵，而是计算批量中每个样本单独计算的偏导数之和

In [51]:
# 对于非标量调用 反向传播函数，需要传入一个 gradient 参数
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

In [52]:
x.grad.zero_()
y = x*x
u = y.detach()
z = u*x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

detach()函数作用？为什么将中间变量作为常数控制梯度流？
作用：
1. 冻结预训练网络（特征提取器）
2. 训练生成对抗网络（GAN）
3. 从计算图中移除中间变量以节省内存

detach 本质上是为了克隆数据层的同时，剥离计算图层——创建出“无历史”的克隆

显式构造控制流的例子：  
体现出**PyTorch 自动微分的核心特性**：计算图在前向传播过程中动态构建，并记录实际执行的操作路径，使得反向传播可以沿着该路径准确计算梯度。

In [53]:
def f(a):
    b = a*2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c
a = torch.randn(size=(),requires_grad=True)
d = f(a)
d.backward()

a.grad == d / a

tensor(True)